# Homework 2: Discovery of Frequent Itemsets and Association Rules

In [1]:
from collections import defaultdict
from itertools import combinations

In [2]:
f=open("T10I4D100K.dat", 'r')

## Frequent Itemsets

In [3]:
#support as percentage of the number of baskets
percentage_support=1
#initialization of the number of baskets in the file
baskets=0 
#initialization of frequent itemsets
itemsets = []                

In [4]:
#returns the items(keys) whose number of occurences(value) is greater than the threshold for the given itemset
def filter_candidates(itemset):
    threshold = (baskets / float(100)) * percentage_support
    return {k: v for k,v in itemset.items() if v >= threshold}

In [5]:
#first step, finds the frequent singletons
singleton_candidates = defaultdict(int)
for basket in f.readlines():
    tuples = basket.split()
    baskets += 1
    for t in tuples:
        singleton_candidates[t] += 1
#filter the candidates and generates the frequent itemset
itemsets.append(filter_candidates(singleton_candidates))
print("Number of baskets: " + str(baskets))
print("Number of unique items in the file: " + str(len(singleton_candidates.items())))
print("Number of frequent singletons: " + str(len(itemsets[0].items())))
print ("Frequent singletons: "+ str(itemsets[0]))

Number of baskets: 100000
Number of unique items in the file: 870
Number of frequent singletons: 375
Frequent singletons: {'25': 1395, '52': 1983, '240': 1399, '274': 2628, '368': 7828, '448': 1370, '538': 3982, '561': 2783, '630': 1523, '687': 1762, '775': 3771, '825': 3085, '834': 1373, '39': 4258, '120': 4973, '205': 3605, '401': 3667, '581': 2943, '704': 1794, '814': 1672, '35': 1984, '674': 2527, '733': 1141, '854': 2847, '950': 1463, '422': 1255, '449': 1890, '857': 1588, '895': 3385, '937': 4681, '964': 1518, '229': 2281, '283': 4082, '294': 1445, '381': 2959, '708': 1090, '738': 2129, '766': 6265, '853': 1804, '883': 4902, '966': 3921, '978': 1141, '104': 1158, '143': 1417, '569': 2835, '620': 2100, '798': 3103, '185': 1529, '214': 1893, '350': 3069, '529': 7057, '658': 1881, '682': 4132, '782': 2767, '809': 2163, '947': 3690, '970': 2086, '227': 1818, '390': 2685, '71': 3507, '192': 2004, '208': 1483, '279': 3014, '280': 2108, '496': 1428, '530': 1263, '597': 2883, '618': 1337

In [6]:
#second step, finds the frequent couples
couple_candidates = defaultdict(int)
f.seek(0)
singletons_set = set(itemsets[0])
#iterates all over the baskets, for each basket generates the combinations of the items already contained in the singletons itemset, to produce only the necessary couples and save computation
#because in general: if X is a frequent k-item set, then all (k-1)-item subsets of X must also be frequent
for basket in f.readlines():
    basket_set = set(basket.split())
    #gets only elements of the current basket with a support greater then the established
    intersection = basket_set.intersection(singletons_set)
    #generates the available couples from previous intersection
    tuples = combinations(sorted(intersection), 2) if len(intersection) >= 2 else list()
    #sum 1 to the counter of each combination
    for t in tuples:
        couple_candidates[t] += 1
#filter the candidates and generates the frequent itemset
itemsets.append(filter_candidates(couple_candidates))
print("Number of unique couple of items in the file: " + str(len(couple_candidates.items())))
print("Number of frequent couples: " + str(len(itemsets[1].items())))
print ("Frequent couples: "+ str(itemsets[1]))

Number of unique couple of items in the file: 70125
Number of frequent couples: 9
Frequent couples: {('39', '704'): 1107, ('39', '825'): 1187, ('704', '825'): 1102, ('227', '390'): 1049, ('789', '829'): 1194, ('368', '829'): 1194, ('217', '346'): 1336, ('368', '682'): 1193, ('390', '722'): 1042}


In [7]:
#splites tuples in single items
def split_tuples(tuples):
    return {item for t in tuples for item in t}

In [8]:
#creates triplets from couples in our case, but is able to create 4-item tuples from triplets etc.
def create_k_itemset(tuples):   
    k_itemset = set()
    #iterates over all the tuples
    for i in range(len(tuples)):
        for j in range(len(tuples)):
            if i < j:
                a = {e for e in tuples[i]}
                b = {e for e in tuples[j]}
                #intersects the elements of the selected tuples 
                intersection = a.intersection(b)
                if len(intersection) == len(a) - 1:
                    #generates the union of the selected tuples
                    union = sorted(a.union(b))
                    k_itemset.add(tuple(union))               
    return k_itemset

In [9]:
#third step, finds the frequent triplets
triplets_candidates = defaultdict(int)
f.seek(0)
couples_set = set(itemsets[1])
couples_split_items = split_tuples(itemsets[1])
for basket in f.readlines():
    basket_set = set(basket.split())
    #gets only elements of the current basket already contained in previous obtained couples (so with a support greater then the established)
    pre_intersection = basket_set.intersection(couples_split_items)
    #generates the available couples from previous intersection
    pre_combs = combinations(sorted(pre_intersection), 2) if len(pre_intersection) >= 2  else list()
    #intersects generated couples with the ones already in couples_set
    intersection = set(pre_combs).intersection(couples_set)
    #generates triplets from couples
    if len(intersection) > 1:
        tuples = create_k_itemset(list(intersection))
        for t in tuples:
            triplets_candidates[t] += 1
#filter the candidates and generates the frequent itemset
itemsets.append(filter_candidates(triplets_candidates))
print("Number of unique triplets of items in the file: " + str(len(triplets_candidates.items())))
print("Number of frequent triplets: " + str(len(itemsets[2].items())))
print ("Frequent triplets: "+ str(itemsets[2]))

Number of unique triplets of items in the file: 4
Number of frequent triplets: 1
Frequent triplets: {('39', '704', '825'): 1035}


## Association Rules

In [10]:
#confidence value
confidence_threshold=0.8
#initialization of associations
associations = dict()

In [11]:
#iterates over the tuples sets to find associations
for i in range(len(itemsets)-1):
    itemset = itemsets[::-1][i]
    for t in itemset.items():
        items = {s for s in t[0]}
        #support of the current tuple
        support = t[1]
        for j in range(len(items))[::-1][:-1]:
            if j == 1:
                combs = items
            else:
                combs = combinations(sorted(items), j)
            for c in combs:
                if j == 1:
                    subsetA = set()
                    subsetA.add(c)
                    #support of the subset
                    supportA = itemsets[0][c]
                else:
                    subsetA = {s for s in c}
                    #support of the subset
                    supportA = itemsets[len(subsetA)-1][tuple(sorted(subsetA))]
                subsetB = sorted(items - subsetA)
                if j == len(items) - 1:
                    subsetB = list(subsetB)[0]
                else:
                    subsetB = tuple(subsetB)
                #computation of the confidence for the selected association rule
                confidence = support / float(supportA)
                if confidence >= confidence_threshold:
                    if len(subsetA) == 1:
                        subsetA = list(subsetA)[0]
                    else:
                        subsetA = tuple(subsetA)
                    string = str(subsetA) + " -> " + str(subsetB)
                    print(string + " : " + str(confidence))
                    associations[string] = confidence            
print("Number of association rules: ",len(associations))

('704', '39') -> 825 : 0.9349593495934959
('825', '39') -> 704 : 0.8719460825610783
('825', '704') -> 39 : 0.9392014519056261
Number of association rules:  3
